In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

INPUT_FILE_PATH = "../output/4_calculate_probability.json"
OUTPUT_FILE_PATH = "../output/5_calculate_causality.csv"

# Various constants with text defined by Abhinav
# These constants are here so that, if Abhinav changes his mind on what the strings will look like, 
# the information is centralized
ALL_ANSWERS = "has all the answers"
ALL_ANSWERS_ROUNDED = "has all the answers when rounded"
SOME_SOLUTION = "has one or more of the answers, but not all of them"
SOME_SOLUTION_ROUNDED = "has one or more of the answers when rounded, but not all of them"
NO_SOLUTION = "says no solution"
INVALID = "invalid"

In [ ]:
data = pd.read_json(INPUT_FILE_PATH)
data

In [ ]:
def get_len(column):
    return column.sum()

def get_valid_len(data):
    return get_len(data["valid"])

def negation(column):
    return 1 - column

def disjunction(column_1, column_2):
    return column_1 | column_2

def conjunction(column_1, column_2):
    return column_1 & column_2

def get_probability(column_1, size):
    return column_1.sum() / size

def is_binary_column(data, column_name):
    return data.apply(lambda row : 0 if (isinstance(row[column_name], int) and (row[column_name] <= 1)) else 1, axis=1).sum() <= 0

def remove_non_binary(data):
    non_binary = []
    for i in data.columns:
        if not is_binary_column(data, i):
            non_binary.append(i)

    return data.drop(columns=non_binary)

def word_similarity(word_1, word_2):
    count = 0
    shortest = min(len(word_1), len(word_2))
    for i in range(0, shortest):
        if word_1[i] == word_2[i]:
            count = count + 1
    return count

In [ ]:
def conditional_probability(occurence, condition):
    if condition.sum() == 0:
        return 0
    return conjunction(occurence, condition).sum() / condition.sum()

def prior(data):
    return conditional_probability(negation(data["is_correct"]), data["valid"])

def discard_non_prima_facie(data):
    name_list = []

    prior_prob = prior(data)
    for cause in data.columns:
        if cause == "valid" or cause == "is_correct":
            continue

        if conditional_probability(negation(data["is_correct"]), data[cause]) <= prior_prob:
            name_list.append(cause)
        
    return name_list

def rel(rel_data, column):

    if prior(rel_data) >= conditional_probability(negation(data["is_correct"]), data[column]):
        return []
        
    if column == "valid" or column == "is_correct":
        return []
    name_list = []

    prior_prob = prior(rel_data)

    for cause in rel_data.columns:
        if cause == "valid" or cause == "is_correct" or cause == column:
            continue
        
        if max(len(column), len(cause)) - word_similarity(cause, column) < 5:
            continue

        difference = conditional_probability(negation(rel_data["is_correct"]), rel_data[cause]) - prior_prob

        if conjunction(rel_data[column], rel_data[cause]).sum() > 0 and difference > 0:
            name_list.append(cause)
    return name_list

def calculate_causality(data, cause):

    if prior(data) >= conditional_probability(negation(data["is_correct"]), data[cause]):
        return "n/a"
        
    relateds = rel(data, cause)

    total_probability = 0
    for related in relateds:
        
        conj = conjunction(data[cause], data[related])
        negj = conjunction(negation(data[cause]), data[related])

        conj = conditional_probability(negation(data["is_correct"]), conj)
        negj = conditional_probability(negation(data["is_correct"]), negj)

        total_probability += (conj - negj)

    if (len(relateds) > 0):
        total_probability /= len(relateds)

    return total_probability

def complete_row(data, cause):
    toReturn = pd.DataFrame({
        "name": [cause], 
        "support": conjunction(data[cause], data["valid"]).sum(),
        "causality": calculate_causality(data, cause),
        "rel": ','.join(rel(data, cause)),
        "conditional_probability":[conditional_probability(negation(data["is_correct"]), data[cause])], 
        "prior": prior(data),
        "conditional - prior": conditional_probability(negation(data["is_correct"]), data[cause]) - prior(data)
    })
    return toReturn

def get_causalities(data):
    toReturn = complete_row(data, "valid")

    for d in data.columns:
        toReturn = toReturn.append(complete_row(data, d))

    return toReturn

In [ ]:
data_causes = remove_non_binary(data)
data_causes = data_causes.query('valid == 1')
# data_causes.drop(discard_non_prima_facie(data_causes), axis=1,inplace=True)
data_causes

In [ ]:
causalities = get_causalities(data_causes)
causalities = causalities.iloc[3:]
causalities.head(40)

In [ ]:
# def find_difference(row):
#     print(row["name"], "-------")
#     for column in data_causes.columns:
#         difference = "n/a"
#         if column != "valid" and column != "is_correct" and column != row["name"]:

#             conj = conjunction(data_causes[row["name"]], data_causes[column])
#             negj = conjunction(negation(data_causes[row["name"]]), data_causes[column])

#             conj = conditional_probability(negation(data_causes["is_correct"]), conj)
#             negj = conditional_probability(negation(data_causes["is_correct"]), negj)

#             difference = conj - negj

#             # print(row["name"], column, difference)
        
#             column_name = "p(c|" + row["name"] + "&" + column + ") - " + "p(~" + row["name"] + "&" + column + ")"
#             row[column_name] = difference
#     display(row)
#     return row

# # find_difference(causalities.iloc[0])
# causalities = causalities.apply(find_difference, axis=1)
# causalities

In [ ]:
causalities.to_csv(OUTPUT_FILE_PATH, index=False)